In [ ]:
import tensorflow as tf
import numpy as np
import Utils.BuildMatrix as bm
import numpy as np

In [ ]:
logDirectory = '/home/jemboo/SharedIpynb/graphs/2'
#tensorboard --logdir=/home/jemboo/SharedIpynb/graphs
#fuser 6006/tcp -k

In [ ]:
%%time
import random
mat_sz = 1000
tf.reset_default_graph()
with tf.Session() as sess:
    with tf.device('/gpu:0'): # swap for 'cpu:0' or whatever
        
        a = tf.constant([random.random() for _ in range(mat_sz * mat_sz)], shape=[mat_sz, mat_sz], name='a')
        b = tf.constant([random.random() for _ in range(mat_sz * mat_sz)], shape=[mat_sz, mat_sz], name='b')
        
        f = tf.matmul(a, b)
        for _ in range(100):
            sess.run(f)

#### Block A

In [ ]:
        aP = tf.random_uniform([1000, 1000], name='aP')
        bP = tf.random_uniform([1000, 1000], name='bP')
        a = tf.Variable(aP, name='a')
        b = tf.Variable(bP, name='b')
        sess.run(a.initializer)
        sess.run(b.initializer)

#### Block B

In [ ]:
        a = tf.constant([random.random() for _ in range(mat_sz * mat_sz)], shape=[mat_sz, mat_sz], name='a')
        b = tf.constant([random.random() for _ in range(mat_sz * mat_sz)], shape=[mat_sz, mat_sz], name='b')

In [ ]:
# Block A, CPU: 5.25334620475769
# Block A, GPU: 0.22746562957763672
# Block B, CPU: 0.9498565196990967
# Block B, GPU: 0.9788544178009033

In [1]:
import random
import tensorflow as tf
import numpy as np

In [6]:
logDirectory = '/home/jemboo/SharedIpynb/graphs/B'
walkLength = 200
mat_sz = 500
tf.reset_default_graph()

In [7]:
k = tf.placeholder(tf.float32)
trajects = tf.Variable(tf.zeros(shape=[mat_sz, mat_sz]))
norm = tf.random_normal(shape=[mat_sz, mat_sz], stddev=k)
prod = tf.matmul(norm, norm)
update = tf.assign_add(trajects, prod)
summary_trajects = tf.summary.histogram('trajects', trajects)
summaries = tf.summary.merge_all()

In [12]:
%%time
with tf.device('/gpu:0'):
    with tf.Session() as sess:
        #writer = tf.summary.FileWriter(logDirectory, sess.graph)
        sess.run(trajects.initializer)
        for i in range(walkLength):
            #trn_out, smms = 
            sess.run([update, summaries], feed_dict={k: 1.0 + float(i)/float(walkLength)})
            #writer.add_summary(smms, global_step=i)

CPU times: user 3.69 s, sys: 96 ms, total: 3.78 s
Wall time: 3.21 s


In [ ]:
CPU times: user 3.94 s, sys: 92 ms, total: 4.03 s
Wall time: 3.45 s

In [ ]:
CPU times: user 3.91 s, sys: 156 ms, total: 4.06 s
Wall time: 3.42 s

In [ ]:
%%time
import random
import tensorflow as tf
import numpy as np

logDirectory = '/home/jemboo/SharedIpynb/graphs/A'
walkLength = 100
d = 2000
tf.reset_default_graph()

#A
trajects = tf.Variable(np.zeros([d, d], dtype=np.float32))
#B
#trajects = tf.Variable(tf.zeros([d, d], dtype=np.float32))
norm = tf.random_normal([d, d], mean=0, stddev=0.04)
update = tf.assign_add(trajects, norm)

summary_trajects = tf.summary.histogram('trajects', trajects)
summaries = tf.summary.merge_all()

with tf.Session() as sess:
    with tf.device('/cpu:0'): # swap for 'cpu:0' or whatever
        writer = tf.summary.FileWriter(logDirectory, sess.graph)

        sess.run(trajects.initializer)

        for i in range(walkLength):
            smms, trn_out = sess.run([summaries, update])
            writer.add_summary(smms, global_step=i)

In [ ]:
# A, CPU: 20.3 s
# A, GPU: 20 s
# B, CPU: 16.7 s
# B, GPU: 16.5 s

In [13]:
tf.reset_default_graph()

In [17]:
dimA = 500
dimB = 500

In [18]:
mAp = tf.random_uniform([dimA, dimB], name='a')
mBp = tf.random_uniform([dimB, dimA], name='b')
mA = tf.Variable(mAp)
mB = tf.Variable(mBp)
mD = tf.matmul(mA, mB)
mE = tf.matrix_inverse(mD)
mF = mB.assign(mE)

In [20]:
%%time
with tf.device("/cpu:0"):
    with tf.Session() as sess:
        #tf.global_variables_initializer().run()
        sess.run(mA.initializer)
        sess.run(mB.initializer)
        for i in range(50):
            prod = sess.run(mF)

CPU times: user 652 ms, sys: 444 ms, total: 1.1 s
Wall time: 2.78 s


In [ ]:
CPU times: user 532 ms, sys: 556 ms, total: 1.09 s
Wall time: 2.77 s

In [ ]:
# Discretized PDE update rules
U_ = mA * mB
Ut_ = Ut + eps * (laplace(U) - damping * Ut)

# Operation to update the state


In [ ]:
with tf.device("/cpu:0"):
    sess = tf.InteractiveSession()
    writer = tf.summary.FileWriter(logDirectory, sess.graph)
    tf.global_variables_initializer().run()
    for i in range(100):
        prod = sess.run(mE)
    sess.close()
    writer.close()

In [ ]:
prod.size

In [ ]:
x = tf.constant(1.0, name='input') 
w = tf.get_variable("weight", 
                    dtype=tf.float32,
                    initializer=tf.constant(value=0.8, name='weight_i'))
y = tf.multiply(w, x, name='output')

In [ ]:
y_ = tf.constant(0.0)
loss = (y - y_)**2

In [ ]:
optim = tf.train.GradientDescentOptimizer(learning_rate=0.05)
grads_and_vars = optim.compute_gradients(loss)

In [ ]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    writer = tf.summary.FileWriter(logDirectory, sess.graph)
    gv = sess.run(optim.apply_gradients(grads_and_vars))
    res = sess.run(w)
    print(res)
writer.close()

In [ ]:
grads_and_vars[0][1]